# Compare room IDs extracted from cad data and the movements (NBEW) database table of the IDP



In [ ]:
import sys
import os
import re

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', None)
import matplotlib.pyplot as plt

first_time = True
if first_time:
    sys.path.append('../')
    os.chdir("../")
    first_time = False

print(os.getcwd())


# SAP Data

## Movement/Stay Data

In [ ]:
base_folder = "./data/interim/model_data/"
movement_df = pd.read_csv(base_folder + "LA_ISH_NBEW.csv", encoding="ISO-8859-1")

sap_nbew_rooms_df = movement_df[["Department", "Ward", "Room ID"]] #, "Bed ID"]]

sap_nbew_rooms_df = sap_nbew_rooms_df.groupby(sap_nbew_rooms_df.columns.tolist()).size().reset_index().rename(columns={0:'count'})
# sap_nbew_rooms_df = sap_nbew_rooms_df.drop_duplicates()

sap_nbew_rooms_df = sap_nbew_rooms_df.fillna(value={"Room ID": "-"})

sap_nbew_rooms_df.rename(columns={"Room ID": "SAP Room ID"}, inplace=True)

sap_nbew_rooms_df.sort_values(by="count", ascending=False) #[sap_rooms_df["Room ID"].str.contains("BH ")]

In [ ]:
# The numbers before the Departments are the following:
# array(['71', nan, '61', '62', '63', '64', '92'], dtype=object)

# sap_nbew_rooms_df["Department"].unique()

# dep_split_df = sap_nbew_rooms_df["Department"].str.extract('([0-9][0-9])?([A-Za-z]{4})', expand=True)
# dep_split_df.columns = ["Building ID?", "Department ID?"]
# sap_nbew_rooms_df = pd.concat([sap_nbew_rooms_df, dep_split_df], axis=1)
# sap_nbew_rooms_df

# sap_nbew_rooms_df["Building ID?"].unique()

In [ ]:
building_split_df = sap_nbew_rooms_df["SAP Room ID"].str.extract('([A-Za-z]+[0-9]*)?\.?[\s]?([A-Za-z]+)?([0-9]\.)?[\s\.-]?([0-9]*[A-Za-z]*)?', expand=True)
building_split_df = pd.concat([building_split_df, sap_nbew_rooms_df["SAP Room ID"]], axis=1)
building_split_df = building_split_df[building_split_df["SAP Room ID"] != "-"]
building_split_df = building_split_df.sort_values(by=["SAP Room ID"]).drop_duplicates()
building_split_df.columns = ["Building ID", "Floor ID", "Floor ID2", "Room ID", "SAP Room ID"]
building_split_df["Floor ID"] = building_split_df['Floor ID'].combine_first(building_split_df['Floor ID2'])
building_split_df.drop(["Floor ID2"], axis=1, inplace=True)
building_split_df

## Room Data

In [ ]:
# NOT SURE WHAT THOSE ROOM IDS ARE (CHECK CODE)
base_folder = "./data/interim/model_data/"
sap_dim_room_df = pd.read_csv(base_folder + "DIM_RAUM.csv",  dtype=str, index_col=0)

sap_dim_room_df = sap_dim_room_df.fillna(value={"Room Common Name": "-"})
#sap_dim_room_df[sap_dim_room_df["Room Common Name"].str.contains("BH")]
sap_dim_room_df.sort_values(by=["Room Common Name"], inplace=True)
sap_dim_room_df

In [ ]:
# NOT SURE WHAT THOSE ROOM IDS ARE (CHECK CODE)
base_folder = "./data/interim/model_data/"
fact_appointment_room_df = pd.read_csv(base_folder + "FAKT_TERMIN_RAUM.csv", encoding="ISO-8859-1", dtype=str)

appointment_rooms = fact_appointment_room_df[["Room ID", "Room Common Name"]]

distinct_rooms = appointment_rooms.drop_duplicates()

distinct_rooms = distinct_rooms.fillna(value={"Room Common Name": "-"})

bh_rooms = distinct_rooms[distinct_rooms["Room Common Name"].str.contains("BH")]

sap_fact_app_room_df = distinct_rooms.sort_values(by="Room Common Name")

sap_fact_app_room_df.set_index("Room ID", inplace=True)

sap_fact_app_room_df

## Building Unit Data

In [ ]:
base_folder = "./data/raw/model_data/"
sap_building_unit_df = pd.read_csv(base_folder + "LA_ISH_NBAU.csv", dtype=str)
# SAP klingon translation: https://www.tcodesearch.com/sap-tables/detail?id=NBAU
sap_building_unit_df.drop(["MANDT", "TELNR", "TELFX", "TELTX", "LOEKZ", "LOUSR", "LODAT",
                           "ERDAT", "ERUSR", "UPDAT", "UPUSR", "BEGDT", "ENDDT", "FREIG",
                           "TALST", "ADDIN","XKOOR", "YKOOR", "BREIT", "LAENG", "ARCHV",
                           "MIGRATED_OBJID", "BATCH_RUN_ID", "ZZBEMK", "ZZVERLEGUNG", "ZZVORHALTE",
                           "ZZPRIVAT", "EANNR", "BETTST_TYP"], axis=1, inplace=True)
sap_building_unit_df.columns = ["SAP Room ID", "Unit Type", "Unit Name", "SAP Room ID 1", "SAP Room ID 2", "Short Text", "Long Text", "Address Information", "Address Object", "Waveware Campus", "Waveware Building ID", "Waveware Floor ID", "Waveware Room ID"]
sap_building_unit_df.set_index("SAP Room ID", inplace=True)
sap_building_unit_df.loc[sap_building_unit_df["Unit Type"] == "Z", "Unit Type"] = "Room"
sap_building_unit_df.loc[sap_building_unit_df["Unit Type"] == "B", "Unit Type"] = "Bettstellplatz"
sap_building_unit_df = sap_building_unit_df[sap_building_unit_df["Unit Type"] == "Room"]
sap_building_unit_df

In [ ]:
def extract_campi(row):
    if pd.isna(row['Waveware Campus']):
        if row['Unit Name'].find("Aarberg") != -1:
            row['Waveware Campus'] = 'AARB'
        if row['Unit Name'].find("Riggisberg") != -1:
            row['Waveware Campus'] = 'RIGG'
        if row['Unit Name'].find("R_") != -1:
            row['Waveware Campus'] = 'RIGG'
        if row['Unit Name'].find("Tiefenau") != -1:
            row['Waveware Campus'] = 'TIEF'
        if row['Unit Name'].find("Münsigen") != -1:
            row['Waveware Campus'] = 'MUEN'
        if row['Unit Name'].find("Belp") != -1:
            row['Waveware Campus'] = 'BELP'

    return row

def extract_ids(row):
    if row['Waveware Campus'] not in ['Aarberg', 'Riggisberg', 'Tiefenau', 'Münsigen'] and not pd.isna(row['SAP Room ID 2']):
        if pd.isna(row["Waveware Room ID"]):
            pattern = '([A-Za-z]+[0-9]*)[\s]+([A-Za-z]*[0-9]*)[\.N\s-]+([0-9]+[A-Za-z]*)'#'([A-Za-z]+[0-9]*)?\.?[\s]?([A-Za-z]+)?([0-9]\.)?[\s\.-]?([0-9]*[A-Za-z]*)?'
            match = re.search(pattern, row['SAP Room ID 2'])
#             if re.search('FK\s*D\s*35', row['SAP Room ID 2']) is not None:
#                 print(row['SAP Room ID 2'])
            if match is not None:
                # if extraction was successful, read metric df and compose tuple
                row['Waveware Building ID'], row['Waveware Floor ID'], row['Waveware Room ID'] = match.groups()
    return row

def extract_waveware_ids(row):
    row = extract_campi(row)
    row = extract_ids(row)
    
    return pd.Series({'Waveware Campus': row['Waveware Campus'], 'Waveware Building ID': row['Waveware Building ID'], 'Waveware Floor ID': row['Waveware Floor ID'], 'Waveware Room ID': row['Waveware Room ID']})

sap_building_unit_fix_df = pd.concat([sap_building_unit_df.drop(["Waveware Campus", "Waveware Building ID", "Waveware Floor ID", "Waveware Room ID"], axis=1), sap_building_unit_df.apply(extract_waveware_ids, axis=1)], axis=1)

print("Campus:", len(sap_building_unit_fix_df[pd.isna(sap_building_unit_fix_df["Waveware Campus"])]))
print("Building", len(sap_building_unit_fix_df[pd.isna(sap_building_unit_fix_df["Waveware Building ID"])]))
print("Floor", len(sap_building_unit_fix_df[pd.isna(sap_building_unit_fix_df["Waveware Floor ID"])]))
print("Room", len(sap_building_unit_fix_df[pd.isna(sap_building_unit_fix_df["Waveware Room ID"])]))

In [ ]:
sap_building_unit_fix_df[['Unit Type', 'Unit Name', 'SAP Room ID 1', 'SAP Room ID 2', 'Waveware Campus', 'Waveware Building ID', 'Waveware Floor ID', 'Waveware Room ID']]

In [ ]:
len(sap_building_unit_fix_df[sap_building_unit_fix_df["Waveware Campus"] == "ISB"])

## Building Data

In [ ]:
base_folder = "./data/raw/model_data/"
sap_building_df = pd.read_csv(base_folder + "LA_ISH_ZHC_RB_BUILDING.csv", dtype=str)
sap_building_df.drop(["MANDT", "ERDAT", "ERNAM", "AEDAT", "AENAM", "BATCH_RUN_ID"], axis=1, inplace=True)
sap_building_df.columns = ["Waveware Campus", "Waveware Building ID", "Building Common Name"]
sap_building_df.set_index("Waveware Building ID", inplace=True)
sap_building_df

## Room Data

In [ ]:
base_folder = "./data/raw/model_data/"
rb_room_df = pd.read_csv(base_folder + "LA_ISH_ZHC_RB_RAUM.csv", dtype=str)
rb_room_df.drop(["MANDT", "ERDAT", "ERNAM", "AEDAT", "AENAM", "BATCH_RUN_ID"], axis=1, inplace=True)
rb_room_df.columns = ["Waveware Campus", "Waveware Building ID", "Waveware Floor ID", "Waveware Room ID", "Room Common Name", "Waveware Room Full ID"]
rb_room_df.set_index("Waveware Room Full ID", inplace=True)
rb_room_df = pd.merge(rb_room_df, sap_building_df, on="Waveware Building ID")
rb_room_df

## Campus Data

In [ ]:
base_folder = "./data/raw/model_data/"
rb_campus_df = pd.read_csv(base_folder + "LA_ISH_ZHC_RB_STANDORT.csv", dtype=str)
rb_campus_df.drop(["MANDT", "ERDAT", "ERNAM", "AEDAT", "AENAM", "BATCH_RUN_ID"], axis=1, inplace=True)
rb_campus_df.columns = ["Waveware Campus", "Common Name"]
rb_campus_df

## Floor Data

In [ ]:
base_folder = "./data/raw/model_data/"
rb_floor_df = pd.read_csv(base_folder + "LA_ISH_ZHC_RB_STOCKWERK.csv", dtype=str)
rb_floor_df.drop(["MANDT", "ERDAT", "ERNAM", "AEDAT", "AENAM", "BATCH_RUN_ID"], axis=1, inplace=True)
rb_floor_df.columns = ["Waveware Campus", "Waveware Building ID", "Waveware Floor ID", "Floor Common Name"]
rb_floor_df = pd.merge(rb_floor_df, sap_building_df, on="Waveware Building ID")
rb_floor_df

# Waveware Data

## Room Data

In [ ]:
base_folder = "./data/raw/cad_data/"
waveware_rooms_df = pd.read_csv(base_folder + "Auszug Flaechenmanagement IDSC (Stand 02.07.20).csv", encoding="ISO-8859-1", dtype=str)

waveware_rooms_df.columns = ["Waveware Building ID", "Building Common Name", "Waveware Floor ID", "Waveware Room ID", "Waveware Room Full ID", "Room Common Name", "Room Area", "PC Group ID", "Sub-EC(PC) Nr", "Profitcenter"]
waveware_rooms_df = waveware_rooms_df.drop(["Room Area", "PC Group ID", "Sub-EC(PC) Nr", "Profitcenter"], axis=1)
# waveware_rooms_df.set_index("Waveware Room Full ID", inplace=True)
waveware_rooms_df

## Building Data

In [ ]:
base_folder = "./data/raw/cad_data/"
waveware_buildings_df = pd.read_csv(base_folder + "Auszug Gebaeudeinformation Stand 03.12.2020.csv", encoding="ISO-8859-1", dtype=str)

waveware_buildings_df = waveware_buildings_df.drop(["Standort", "Parzellennummer", "Zonenplan", "Denkmalpflege", "Anlage-ID", "Bemerkung", "Eigentümer (SAP)", "Vermietung (SAP)", "Portfolio (SAP)", "Baujahr", "Gebäudetyp", "GVB-Nummer", "Amtlicher Wert", "Gebäudeversicherungswert", "Gebäudezustand", "Technologiestand HLKSE", "Techn. Ausb.standard", "Zustand Technik", "Klimatisierung", "Aufzug", "Gebäudezustand Bem.", "Status"], axis=1)
waveware_buildings_df.columns = ["Waveware Building Full ID", "Building Code", "Waveware Building ID", "Building abbreviation", "Building Common Name", "Street", "Zip Code", "Location", "SAP-Anlage Nr."]
waveware_buildings_df.drop(["Zip Code", "Location","SAP-Anlage Nr.", "Building Code"], axis=1, inplace=True)
waveware_buildings_df = waveware_buildings_df[waveware_buildings_df["Building Common Name"] != "Grundstück Inselareal"]

waveware_buildings_df = waveware_buildings_df[~pd.isna(waveware_buildings_df["Building abbreviation"])]
waveware_buildings_df.sort_values(by=["Building abbreviation"], inplace=True)
# waveware_buildings_df.set_index("Waveware Building ID", inplace=True)
waveware_buildings_df

## Location data (Open Street Map)

In [ ]:
import requests

def get_long_lat(street_string):
    response = requests.get(f"https://nominatim.openstreetmap.org/search?q={street_string.replace(' ', '+')}+Bern&format=json")
    types = []
    for loc in response.json():
        types.append(loc["type"] + ": " + loc["display_name"][:15])
        if loc["type"] in ["hospital", "childcare", "clinic"]:
            id_string = loc["type"] + ": " + loc["display_name"][:15]
            long_lat = (loc["lon"], loc["lat"])
            return pd.Series({'Type': id_string, 'Long/Lat': long_lat})
        
    id_string = response.json()[0]["type"] + ": " + response.json()[0]["display_name"][:15]
    long_lat = (response.json()[0]["lon"], response.json()[0]["lat"])
    return pd.Series({'Type': id_string, 'Long/Lat': long_lat})


waveware_buildings_coords_df = pd.concat([waveware_buildings_df, waveware_buildings_df["Street"].apply(lambda s: get_long_lat(s))], axis=1)
waveware_buildings_coords_df["Longitude"] = waveware_buildings_coords_df["Long/Lat"].apply(lambda ll: float(ll[0]))
waveware_buildings_coords_df["Latitude"] = waveware_buildings_coords_df["Long/Lat"].apply(lambda ll: float(ll[1]))
waveware_buildings_coords_df.drop(["Long/Lat"], axis=1, inplace=True)
waveware_buildings_coords_df

In [ ]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import contextily as ctx
gdf = geopandas.GeoDataFrame(
    waveware_buildings_coords_df, geometry=geopandas.points_from_xy(waveware_buildings_coords_df.Longitude, waveware_buildings_coords_df.Latitude))
gdf.set_crs(epsg=4326, inplace=True)
gdf = gdf.to_crs(epsg=3857)

gdf

In [ ]:
ax = gdf.plot(figsize=(16, 12), alpha=0.5, edgecolor='k', label="Building abbreviation")
ctx.add_basemap(ax, zoom=15, source=ctx.providers.CartoDB.Positron, alpha=0.5) # OpenStreetMap.Mapnik, CartoDB.Positron, CartoDB.Voyager
# providers: https://contextily.readthedocs.io/en/latest/providers_deepdive.html

gdf['coords'] = gdf['geometry'].apply(lambda x: x.representative_point().coords[:])
gdf['coords'] = [coords[0] for coords in gdf['coords']]

for idx, row in gdf.iterrows():
    plt.annotate(s=row['Building abbreviation'], xy=row['coords'], horizontalalignment='center', verticalalignment='bottom')
plt.show()

## CAD Data

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import re

# get all label csv files
label_csv_files = list(Path("./data/processed/cad_maps/").glob("**/*_labels.csv"))

pbar = tqdm(label_csv_files)
floor_dfs = []
for csv_file in pbar:
    csv_file_path = str(csv_file)
    pbar.set_description(f"Processing {csv_file_path}...")

    floor_labels = pd.read_csv(csv_file_path, index_col=0)
    floor_labels = floor_labels[floor_labels["Label Type"] == "Room ID"]
    floor_dfs.append(floor_labels)

cad_rooms_df = pd.concat(floor_dfs)

cad_rooms_df.reset_index()

cad_rooms_df = cad_rooms_df[["Building ID", "Floor ID", "Label Text"]]

cad_rooms_df.rename(columns={"Building ID": "Building Nr", "Label Text": "Room ID"}, inplace=True)

cad_rooms_df.sort_values(by=["Building Nr"], inplace=True)

cad_rooms_df

# Merges

In [ ]:
sap_nbew_rooms_df.head()

In [ ]:
sap_dim_room_df.head()

In [ ]:
sap_fact_app_room_df.head()

In [ ]:
waveware_rooms_df.head()

In [ ]:
waveware_buildings_df.head()

In [ ]:
cad_rooms_df.head()


In [ ]:
pd.merge(rb_room_df, waveware_rooms_df, on="Waveware Building ID")

In [ ]:
len(waveware_rooms_df)

In [ ]:
sap_building_fix_isb_df = sap_building_unit_fix_df[sap_building_unit_fix_df["Waveware Campus"] == "ISB"]
waveware_sap_merge = pd.merge(waveware_rooms_df, sap_building_fix_isb_df, on=["Waveware Building ID", "Waveware Floor ID", "Waveware Room ID"])
waveware_sap_merge[["Waveware Building ID", "Building Common Name", "Waveware Floor ID", "Waveware Room ID", "Waveware Room Full ID", "Room Common Name", "SAP Room ID 1", "SAP Room ID 2"]]

In [ ]:
len(waveware_sap_merge)

In [ ]:
waveware_sap_merge_fails = pd.merge(waveware_rooms_df, sap_building_fix_isb_df, how='outer', on=["Waveware Building ID", "Waveware Floor ID", "Waveware Room ID"], indicator=True)

In [ ]:
waveware_sap_merge_fails = waveware_sap_merge_fails[waveware_sap_merge_fails["_merge"] != "both"]
excluded_buildings = ["Unterirdische Verbindungsgänge", "Sitem-Insel", "Schwarztorstrasse 96",
                      "Geschäftshaus Güterstrasse 24/26", "Geschäftshaus Effingerstrasse 77",
                     "Wohnhaus Lentulusrain 11", "Wohnhaus Hopfenrain 16", "Wohnhaus Zieglerstrasse 8",
                      "Wohnhaus Freiburgstrasse 65", "Wohnhaus Murtenstrasse 34", "Wohnhaus Murtenstrasse 40/42",
                      "Murtenstrasse 50", "Murtenstrasse 10", "Apotheke", "Dekanat", "Wirtschaftsgebäude",
                     "Lagerhaus Güterstrasse 13"]
excluded_rooms = ["WC", "WC/Du", "Bad/Du", "WC Pers.", "WC Inv.", "Bad", "Materialraum", "Aufenthalt", "Windfang", "Heizung/Sanitär", "Lüftung",
                  "Mehrzweckraum",
                  "Garderobe Damen", "Garderobe Herren", "Apotheke", "Korridor", "Lift", "Treppe", "Putzraum", "Lager", "Velos", "Sanitär"]
waveware_sap_merge_fails = waveware_sap_merge_fails[~waveware_sap_merge_fails["Building Common Name"].isin(excluded_buildings)]
waveware_sap_merge_fails = waveware_sap_merge_fails[~waveware_sap_merge_fails["Room Common Name"].isin(excluded_rooms)]
waveware_sap_merge_fails_show = waveware_sap_merge_fails[["_merge", "Waveware Building ID", "Building Common Name", "Waveware Floor ID", "Waveware Room ID", "Waveware Room Full ID", "Room Common Name", "SAP Room ID 1", "SAP Room ID 2"]]
waveware_sap_merge_fails_show

In [ ]:
len(waveware_sap_merge_fails_show)

## New idea
sap_nbew_rooms_df has a Room ID that seems to be splittable such that the first part looks like waveware Building abbreviations


In [ ]:
waveware_rooms_df2 = pd.merge(waveware_rooms_df, waveware_buildings_df, on="Waveware Building ID", suffixes=("","_y"))
waveware_rooms_df2.drop(["Building Common Name_y"], axis=1, inplace=True)
waveware_rooms_df2